# YOLOV4 OBJECT DETECTION

### STEP 1: DARKNET CLONE AND INSTALLATION

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile  #I will use opencv library in darknet
!sed -i 's/GPU=0/GPU=1/' Makefile #I will use cpu library in darknet
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile #cudnn a necessary tool for cpu
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

### STEP 2: DEFINING AUXILIARY FUNCTIONS

In [ ]:
# define helper functions
def imShow(path): 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

### STEP 4: LOADING THE DATASET WE HAVE PREPARED

In [ ]:
# this is where my datasets are stored within my Google Drive (I created a yolov4 folder to store all important files for custom training) 
%cd darknet/
!ls /mydrive/yolov4

In [ ]:
# copy over both datasets into the root directory of the Colab VM (comment out test.zip if you are not using a validation dataset)
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

### STEP 5: LET'S PREPARE THE REQUIRED FILES FOR THE TRAINING

In this step, we will create the .cfg file, obj.data, obj.names and train.txt files required for the training.

download cfg to google drive and change its name                         
```
from shutil import copy2                                    
copy2("/content/darknet/cfg/yolov4-custom.cfg","/mydrive/yolov4")    
```      
you can also copy like this


In [ ]:
# download cfg to google drive and change its name
#%cd darknet/
!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-obj.cfg

Changes we need to make in our config file:

(The values given here are the recommended values for these variables.)

1. batch = 64 and subdivision 16.

3. We equalize max_batches (2000 * number of classes trained). But the minimum we can do is 6000, so if you have one, two and three classes, it should be 6000.

4. We make the values of the steps (80% of max_batches), (90% of max_batches).

5. We replace the classes values under the [yolo] heading with the number of classes we train.

6. We also equalize the filter's variables (number of classes to train + 5 )*3

We upload our updated file from the drive

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive

!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

*-obj.names and obj.data*

Let's create a file named obj.names in our folder named yolov3 and write the names of your objects that we will train the file with.

E.G:
```
traffic sign
traffic light
```

In the same folder, we create a file with the name obj.data and write the directory where we will save the number of objects we will train, the addresses of the files named train.txt, text.txt and obj.names that we will use while training, and the weights we find as a result of the training.

E.G:
```
classes = 1
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /mydrive/yolov4/backup
```


upload the obj.names and obj.data files to cloud VM from Google Drive

In [ ]:

!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

**-Train and Test Files**


generate_train.py



```
import os
image_files = []
os.chdir(os.path.join("data", "obj"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/obj/" + filename)
os.chdir("..")
with open("train.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```

generate_test.py



```
import os

image_files = []
os.chdir(os.path.join("data", "test"))
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".jpg"):
        image_files.append("data/test/" + filename)
os.chdir("..")
with open("test.txt", "w") as outfile:
    for image in image_files:
        outfile.write(image)
        outfile.write("\n")
    outfile.close()
os.chdir("..")
```


upload the generate_train.py and generate_test.py script to cloud VM from Google Drive


In [ ]:
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

### STEP 6: REDUCE THE WEIGHTS OF PRE-TRAINED CONVOLUTIONAL LAYERS

In this step, we download the used deep learning layers weights for the pre-trained yolov3. We do not have to perform this step, but starting the training with these weights will help the model we train to work more accurately and shorten the training time.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

### STEP 7: TRAINING
All the necessary files are ready, we can start the training.

Training will begin with the next command.

The duration of our training may vary depending on factors such as the number of photos in your data set, the quality of the photos, and the number of objects you train. Our loss value is important for the accuracy of our model. The lower our Loss value, the more accurate our model will work. We can run our model until the loss value stops decreasing and train the most accurate model possible according to our data set.

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# the graph of our training.
imShow('chart.png')

If we don't like the weights, we can continue the training from where we left off.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

### STEP 7: USE OUR TRAINED MODEL


Our training is complete, now we can make recognition on the photos we want.

Let's look at the average loss value and percent accuracy of our model

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights

**Let's run our model**

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

To predict and see individual images

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/2021-08-20_15-18.png -thresh 0.3
imShow('predictions.jpg')

**To predict and save multiple images**

In [ ]:
!cp /mydrive/yolov4/test_pred.zip ../


In [ ]:
%cd /content/

In [ ]:
!unzip test_pred.zip 

In [ ]:
%cd /content/darknet/

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:

def predictImage(image_path):
    import glob
    import os
    import cv2
    import matplotlib.pyplot as plt
    image_path_list = glob.glob(os.path.join(image_path, '*'))
    image_path_list.sort()
    for imageDir in image_path_list:
      !./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights imageDir -thresh 0.3
      image = cv2.imread("/content/darknet/predictions.jpg")
      height, width = image.shape[:2]
      resized_image = cv2.resize(image,(3*width, 3*height),interpolation = cv2.INTER_CUBIC)
      predict_path='/mydrive/yolov4/predict_image/'+imageDir.split('/')[1]
      fig = plt.gcf()
      fig.set_size_inches(15, 8)
      plt.axis("off")
      image=plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
      cv2.imwrite(predict_path,image)
predictImage('test_pred')
  
  